In [58]:
from http.client import HTTPException
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [72]:
import pandas as pd
import requests
import json
import pprint


# Testing HL Api for PULL transfer

---

## Consts

In [92]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
api_endpoint = "https://jsonplaceholder.typicode.com/posts"

catalog_id = ""
dataservice_id = ""
agreement_id = ""

## Setup Catalog, Dataservice and agreements

In [93]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'urn:uuid:2e08b9c1-d1ac-4126-b014-637737f12722',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'urn:uuid:2e08b9c1-d1ac-4126-b014-637737f12722',
 'dct:issued': '2025-01-27T14:52:54.130603',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [94]:
payload = {
    "dcat:endpointURL": api_endpoint
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'urn:uuid:ed1fb377-05a4-4168-ad92-0b5d3ca91afd',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'https://jsonplaceholder.typicode.com/posts',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'urn:uuid:ed1fb377-05a4-4168-ad92-0b5d3ca91afd',
 'dct:issued': '2025-01-27T14:52:54.591195',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [95]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]

pprint.pprint(response.json())

{'agreement_id': 'urn:uuid:56cc6cd2-f87c-4427-b3ce-e5e93fef2f6a',
 'data_service_id': 'urn:uuid:ed1fb377-05a4-4168-ad92-0b5d3ca91afd',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [102]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://127.0.0.1:1235/urn:uuid:3789cb32-3c26-4a6b-b687-6cb3079a1b99',
 'callbackId': 'urn:uuid:3789cb32-3c26-4a6b-b687-6cb3079a1b99',
 'consumerPid': 'urn:uuid:a10b4ed3-2dd6-4ef9-93a4-452e9af26db3'}


## Request transfer

In [103]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_id,
  "format": "http+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  #"callbackAddress": "",
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:a10b4ed3-2dd6-4ef9-93a4-452e9af26db3',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:a10b4ed3-2dd6-4ef9-93a4-452e9af26db3',
                     'dspace:providerPid': 'urn:uuid:adce0367-1dd7-4034-a42b-12a94f45f09e',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [83]:
import requests

url = data_space_consumer + "/api/v1/data-address/" + consumerPid

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://127.0.0.1:1235/urn:uuid:30c1f981-2c9b-414b-beda-767235e1c601/data/pull/urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd'}


## Use data address to get data from data space

In [84]:
df = pd.read_json(dataPlaneAddress)
df

,userId,id,title,body
0,1,1,sunt aut facere repellat provident occaecati e...,quia et suscipit\nsuscipit recusandae consequu...
1,1,2,qui est esse,est rerum tempore vitae\nsequi sint nihil repr...
2,1,3,ea molestias quasi exercitationem repellat qui...,et iusto sed quo iure\nvoluptatem occaecati om...
3,1,4,eum et est occaecati,ullam et saepe reiciendis voluptatem adipisci\...
4,1,5,nesciunt quas odio,repudiandae veniam quaerat sunt sed\nalias aut...
...,...,...,...,...
95,10,96,quaerat velit veniam amet cupiditate aut numqu...,in non odio excepturi sint eum\nlabore volupta...
96,10,97,quas fugiat ut perspiciatis vero provident,eum non blanditiis soluta porro quibusdam volu...
97,10,98,laboriosam dolor voluptates,doloremque ex facilis sit sint culpa\nsoluta a...
98,10,99,temporibus sit alias delectus eligendi possimu...,quo deleniti praesentium dicta non quod\naut e...


## Suspend transfer

In [85]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
                     'dspace:providerPid': 'urn:uuid:82bbe1c9-0a0c-43c6-9c8f-62d3dfcc701e',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [86]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

try:
    df = pd.read_json(url)
    print(df)
except RuntimeError:
    print("asdsad")

HTTPError: HTTP Error 400: Bad Request

## Restart transfer

In [87]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
                     'dspace:providerPid': 'urn:uuid:82bbe1c9-0a0c-43c6-9c8f-62d3dfcc701e',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [88]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,userId,id,title,body
0,1,1,sunt aut facere repellat provident occaecati e...,quia et suscipit\nsuscipit recusandae consequu...
1,1,2,qui est esse,est rerum tempore vitae\nsequi sint nihil repr...
2,1,3,ea molestias quasi exercitationem repellat qui...,et iusto sed quo iure\nvoluptatem occaecati om...
3,1,4,eum et est occaecati,ullam et saepe reiciendis voluptatem adipisci\...
4,1,5,nesciunt quas odio,repudiandae veniam quaerat sunt sed\nalias aut...
...,...,...,...,...
95,10,96,quaerat velit veniam amet cupiditate aut numqu...,in non odio excepturi sint eum\nlabore volupta...
96,10,97,quas fugiat ut perspiciatis vero provident,eum non blanditiis soluta porro quibusdam volu...
97,10,98,laboriosam dolor voluptates,doloremque ex facilis sit sint culpa\nsoluta a...
98,10,99,temporibus sit alias delectus eligendi possimu...,quo deleniti praesentium dicta non quod\naut e...


## Complete transfer

In [89]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
                     'dspace:providerPid': 'urn:uuid:82bbe1c9-0a0c-43c6-9c8f-62d3dfcc701e',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [90]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

HTTPError: HTTP Error 400: Bad Request

## Protocol state cannot change

In [91]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:0f85b5e2-4aa0-4a9c-a9fd-12770bb097bd',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
